In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Kolmogorov-Arnold Neural Network Implementation in PyTorch
class KolmogorovArnoldNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_summation_terms):
        """
        Initialize the Kolmogorov-Arnold Neural Network.
        Args:
            input_dim: Number of input variables (dimensionality of the input).
            hidden_dim: Number of hidden neurons in each univariate sub-network.
            output_dim: Number of outputs (dimensionality of the output).
            num_summation_terms: Number of summation terms in the decomposition.
        """
        super(KolmogorovArnoldNet, self).__init__()

        # Univariate networks: One for each summation term
        self.univariate_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),  # Fully connected layer
                nn.ReLU(),                        # Activation function
                nn.Linear(hidden_dim, 1)          # Map to a scalar output
            )
            for _ in range(num_summation_terms)
        ])

        # Output weights for combining the summation terms
        self.output_weights = nn.Parameter(torch.randn(num_summation_terms, output_dim))

    def forward(self, x):
        """
        Forward pass through the network.
        Args:
            x: Input tensor of shape (batch_size, input_dim).
        Returns:
            Output tensor of shape (batch_size, output_dim).
        """
        # Compute outputs from each univariate network
        summation_terms = torch.cat([net(x) for net in self.univariate_nets], dim=1)

        # Linearly combine summation terms using output weights
        output = summation_terms @ self.output_weights
        return output


# Define the synthetic target function (for demonstration purposes)
def target_function(x):
    """
    Target multivariate function to approximate.
    Args:
        x: Input tensor of shape (batch_size, input_dim).
    Returns:
        Tensor of shape (batch_size, 1) with function values.
    """
    # Example: A nonlinear function of multiple variables
    return torch.sin(x[:, 0]*x[:, 1]) + (torch.cos(x[:, 1])**2)*torch.sin(x[:,2]) + 0.5 * x[:, 2]**2

# Parameters of the net 
input_dim=3
num_samples = 100000
learning_rate = 0.0005
epochs=20000

# Generate synthetic data
x = torch.rand(num_samples, input_dim) * 2 - 1  # Random values in [-1, 1]
y = target_function(x).unsqueeze(1)  # Compute target values and add output dimension


# Creating list of epochs
L1 = list(range(1, epochs + 1))
Val = []
Val1 = []


# Training the Kolmogorov-Arnold Neural Network
def train_kann(x,y,num_samples):
    """
    Train the Kolmogorov-Arnold Neural Network to approximate the target function.
    """
    # Parameters
    #input_dim = 3          # Number of input variables
    hidden_dim = 10        # Hidden layer size in each sub-network
    output_dim = 1         # Number of outputs
    num_summation_terms = 10  # Number of summation terms in the decomposition
    batch_size = 32        # Training batch size
    #epochs = 10000           # Number of training epochs
    #learning_rate = 0.005   # Learning rate

    # Split into training and validation sets
    train_size = int(0.5 * num_samples)
    x_train, y_train = x[:train_size], y[:train_size]
    x_val, y_val = x[train_size:], y[train_size:]

    # Initialize the network
    kann = KolmogorovArnoldNet(input_dim, hidden_dim, output_dim, num_summation_terms)

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(kann.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        # Training step
        kann.train()
        optimizer.zero_grad()
        y_pred = kann(x_train)
        train_loss = criterion(y_pred, y_train)
        train_loss.backward()
        optimizer.step()

        # Validation step
        kann.eval()
        with torch.no_grad():
            y_val_pred = kann(x_val)
            val_loss = criterion(y_val_pred, y_val)
            Val.append(val_loss)

        # Print progress
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss.item():.6f}, Validation Loss: {val_loss.item():.6f}")

    # Return the trained network
    return kann


# Train the network
if __name__ == "__main__":
    trained_kann = train_kann(x,y,num_samples)

    # Test on new data
    test_data = torch.tensor([[0.1, -0.2, 0.3]])
    with torch.no_grad():
        prediction = trained_kann(test_data)
    print(f"Prediction for input {test_data.numpy()}: {prediction.numpy()}")


Epoch 1/20000, Training Loss: 6.061600, Validation Loss: 5.950627
Epoch 2/20000, Training Loss: 5.951417, Validation Loss: 5.841536
Epoch 3/20000, Training Loss: 5.842445, Validation Loss: 5.733677
Epoch 4/20000, Training Loss: 5.734697, Validation Loss: 5.627065
Epoch 5/20000, Training Loss: 5.628195, Validation Loss: 5.521712
Epoch 6/20000, Training Loss: 5.522947, Validation Loss: 5.417631
Epoch 7/20000, Training Loss: 5.418969, Validation Loss: 5.314833
Epoch 8/20000, Training Loss: 5.316270, Validation Loss: 5.213325
Epoch 9/20000, Training Loss: 5.214858, Validation Loss: 5.113114
Epoch 10/20000, Training Loss: 5.114741, Validation Loss: 5.014210
Epoch 11/20000, Training Loss: 5.015927, Validation Loss: 4.916619
Epoch 12/20000, Training Loss: 4.918422, Validation Loss: 4.820345
Epoch 13/20000, Training Loss: 4.822231, Validation Loss: 4.725395
Epoch 14/20000, Training Loss: 4.727359, Validation Loss: 4.631769
Epoch 15/20000, Training Loss: 4.633809, Validation Loss: 4.539471
Epoc

In [ ]:

# Conventional Neural Network Implementation in PyTorch
class ConventionalNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers):
        """
        Initialize a conventional fully connected neural network.
        Args:
            input_dim: Number of input variables (dimensionality of the input).
            hidden_dim: Number of neurons in each hidden layer.
            output_dim: Number of outputs (dimensionality of the output).
            num_hidden_layers: Number of hidden layers in the network.
        """
        super(ConventionalNN, self).__init__()
        
        # Create the input layer
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        
        # Add hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        # Add the output layer
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        # Register the network layers
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass through the network.
        Args:
            x: Input tensor of shape (batch_size, input_dim).
        Returns:
            Output tensor of shape (batch_size, output_dim).
        """
        return self.network(x)


# Define the synthetic target function (for demonstration purposes)
def target_function(x):
    """
    Target multivariate function to approximate.
    Args:
        x: Input tensor of shape (batch_size, input_dim).
    Returns:
        Tensor of shape (batch_size, 1) with function values.
    """
    # Example: A nonlinear function of multiple variables
    return torch.sin(x[:, 0]*x[:, 1]) + (torch.cos(x[:, 1])**2)*torch.sin(x[:,2]) + 0.5 * x[:, 2]**2


# Training the Conventional Neural Network
def train_conventional_nn():
    """
    Train a conventional fully connected neural network to approximate the target function.
    """
    # Parameters
    #input_dim = 3          # Number of input variables
    hidden_dim = 20        # Hidden layer size
    output_dim = 1         # Number of outputs
    num_hidden_layers = 3  # Number of hidden layers
    batch_size = 32        # Training batch size
    #epochs = 10000           # Number of training epochs
    #learning_rate = 0.01   # Learning rate


    # Split into training and validation sets
    train_size = int(0.5 * num_samples)
    x_train, y_train = x[:train_size], y[:train_size]
    x_val, y_val = x[train_size:], y[train_size:]
    
    # Initialize the network
    nn_model = ConventionalNN(input_dim, hidden_dim, output_dim, num_hidden_layers)

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn_model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        # Training step
        nn_model.train()
        optimizer.zero_grad()
        y_pred = nn_model(x_train)
        train_loss = criterion(y_pred, y_train)
        train_loss.backward()
        optimizer.step()

        # Validation step
        nn_model.eval()
        with torch.no_grad():
            y_val_pred = nn_model(x_val)
            val_loss = criterion(y_val_pred, y_val)
            Val1.append(val_loss)

        # Print progress
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss.item():.6f}, Validation Loss: {val_loss.item():.6f}")

    # Return the trained network
    return nn_model


# Train the network
if __name__ == "__main__":
    trained_nn_model = train_conventional_nn()

    # Test on new data
    test_data = torch.tensor([[0.1, -0.2, 0.3]])
    with torch.no_grad():
        prediction = trained_nn_model(test_data)
    print(f"Prediction for input {test_data.numpy()}: {prediction.numpy()}")

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(L1, Val, marker='.', linestyle='-', color='b', linewidth=0.5, markersize=0.5, label='KANN')
plt.plot(L1, Val1, marker='.', linestyle='-', color='r', linewidth=0.5, markersize=0.5, label='Conv. NN')

# Add labels and title

plt.yscale('log')
plt.xlabel('epochs')
plt.ylabel('Validation error')
plt.title('Graph of Validation error over epochs')
plt.legend()

# Show the graph
plt.grid(True)
plt.show()